# CHSH Introspective Runs with Scheduler Circuits

Este cuaderno demuestra como usar **Qonscious** para evaluar la figura de merito CHSH en un backend simulado y, en funcion del resultado, ejecutar algunos circuitos del directorio Scheduler/circuits-code.

## Configuracion

Importamos los modulos de Qonscious y definimos algunas utilidades para construir los circuitos clasicos del repositorio Scheduler/circuits-code sin necesidad de ejecutar los scripts originales (que dependen de proveedores externos).

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Callable

from IPython.display import display
from qiskit import QuantumCircuit

try:
    from qiskit.visualization import plot_histogram
except (ImportError, RuntimeError):
    plot_histogram = None

from qonscious.adapters.aer_sampler_adapter import AerSamplerAdapter
from qonscious.actions.qonscious_circuit import QonsciousCircuit
from qonscious.actions.qonscious_callable import QonsciousCallable
from qonscious.checks.merit_compliance_check import MeritComplianceCheck
from qonscious.foms.packed_chsh import PackedCHSHTest
from qonscious.run_conditionally import run_conditionally

## Circuitos adaptados desde Scheduler/circuits-code

Cada funcion reproduce la logica de los scripts originales, pero enfocandose unicamente en la construccion del circuito.

In [ ]:
def build_grover_circuit() -> QuantumCircuit:
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.h(1)
    qc.x(1)
    qc.h(1)
    qc.cx(0, 1)
    qc.h(1)
    qc.x(1)
    qc.h(1)
    qc.measure([0, 1], [0, 1])
    return qc


def build_qwalk_circuit() -> QuantumCircuit:
    qc = QuantumCircuit(3, 3)
    qc.h(2)
    qc.cx(2, 1)
    qc.ccx(2, 1, 0)
    qc.x(2)
    qc.cx(2, 1)
    qc.x(1)
    qc.ccx(2, 1, 0)
    qc.x(1)
    qc.x(2)
    qc.cx(2, 1)
    qc.ccx(2, 1, 0)
    qc.x(2)
    qc.cx(2, 1)
    qc.x(1)
    qc.ccx(2, 1, 0)
    qc.x(1)
    qc.x(2)
    qc.measure([0, 1, 2], [0, 1, 2])
    return qc


def build_qft_circuit(n_qubits: int = 3) -> QuantumCircuit:
    qc = QuantumCircuit(n_qubits, n_qubits)
    for j in range(n_qubits):
        qc.h(j)
        for k in range(j + 1, n_qubits):
            qc.cp(3.141592653589793 / (2 ** (k - j)), k, j)
    qc.measure(range(n_qubits), range(n_qubits))
    return qc


def draw_circuit(circuit: QuantumCircuit):
    try:
        return circuit.draw("mpl")
    except (ModuleNotFoundError, ImportError):
        return circuit.draw("text")

## Helper: ejecutar un circuito controlado por CHSH

Definimos una accion de fallback (cuando el CHSH no supera el umbral) y una pequena funcion para correr cada circuito bajo 
un_conditionally.

In [ ]:
THRESHOLD = 2.0

backend = AerSamplerAdapter()

check = MeritComplianceCheck(
    figure_of_merit=PackedCHSHTest(),
    decision_function=lambda result: result is not None and result["properties"]["score"] > THRESHOLD,
)

fallback = QonsciousCallable(
    lambda adapter, fom_results, **kwargs: None,
)


def run_under_chsh(builder: Callable[[], QuantumCircuit], shots: int = 1024):
    circuit = builder()
    result = run_conditionally(
        backend_adapter=backend,
        checks=[check],
        on_pass=QonsciousCircuit(circuit),
        on_fail=fallback,
        shots=shots,
    )
    return circuit, result

## Ejecutar y visualizar resultados

Probamos algunos circuitos y mostramos tanto el valor de la figura de merito como la distribucion de resultados.

In [ ]:
@dataclass
class CircuitRun:
    name: str
    circuit: QuantumCircuit
    qonscious_result: dict


runs = []
for name, builder in [
    ("Grover", build_grover_circuit),
    ("Quantum Walk", build_qwalk_circuit),
    ("QFT (3 qubits)", lambda: build_qft_circuit(3)),
]:
    circuit, result = run_under_chsh(builder)
    runs.append(CircuitRun(name, circuit, result))
    fom = result["figures_of_merit_results"][0]
    print(f"{name}: CHSH score = {fom['properties']['score']:.3f}, condition = {result['condition']}")

In [ ]:
for run in runs:
    counts = run.qonscious_result["experiment_result"]
    display(draw_circuit(run.circuit))
    if counts is not None:
        if plot_histogram is not None:
            display(plot_histogram(counts["counts"], title=f"{run.name} counts"))
        else:
            print(f"{run.name} counts: {counts['counts']}")
    else:
        print(f"{run.name}: ejecucion omitida (fallback)")

## Observaciones

- El test CHSH (PackedCHSHTest) se ejecuta antes de cada circuito y determina si la condicion de calidad es suficiente.
- Si la metrica supera el umbral, 
un_conditionally ejecuta el circuito correspondiente usando AerSamplerAdapter.
- La estructura se puede extender para probar mas circuitos o para integrar backends reales simplemente cambiando el adaptador y las credenciales necesarias.